In [1]:
from mlflow.tracking import MlflowClient
MLFLOW_TRACKING_URI="http://localhost:5000"

client= MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

In [2]:
client.search_experiments()

[<Experiment: artifact_location='mlflow-artifacts:/2', creation_time=1718200446766, experiment_id='2', last_update_time=1718200446766, lifecycle_stage='active', name='my-cool-experiment', tags={}>,
 <Experiment: artifact_location='mlflow-artifacts:/1', creation_time=1718182619426, experiment_id='1', last_update_time=1718182619426, lifecycle_stage='active', name='nyc-taxi', tags={}>,
 <Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1718182425008, experiment_id='0', last_update_time=1718182425008, lifecycle_stage='active', name='Default', tags={}>]

In [4]:
#client.create_experiment(name='my-cool-experiment')

'2'

In [3]:
from mlflow.entities import ViewType

runs = client.search_runs(
    experiment_ids='1',
    filter_string="metrics.rmse < 6.9",
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=5,
    order_by=["metrics.rmse ASC"]
)

In [4]:
for run in runs:
    print(f"run id: {run.info.run_id}, rmse: {run.data.metrics['rmse']:.4f}")

run id: b4239ac133884f65936298bd9d2af4bc, rmse: 6.3184
run id: c409406a4bac42be9c14ab22f67c59a9, rmse: 6.7423
run id: f783f99b110e448eb0d509b2f2d09953, rmse: 6.8995


In [5]:
import mlflow

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

In [6]:
run_id = "f783f99b110e448eb0d509b2f2d09953"
model_uri = f"runs:/{run_id}/model"
mlflow.register_model(model_uri=model_uri, name="nyc-taxi-regressor")

Registered model 'nyc-taxi-regressor' already exists. Creating a new version of this model...
2024/06/13 11:13:19 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation. Model name: nyc-taxi-regressor, version 3
Created version '3' of model 'nyc-taxi-regressor'.


<ModelVersion: aliases=[], creation_timestamp=1718266398992, current_stage='None', description='', last_updated_timestamp=1718266398992, name='nyc-taxi-regressor', run_id='f783f99b110e448eb0d509b2f2d09953', run_link='', source='mlflow-artifacts:/1/f783f99b110e448eb0d509b2f2d09953/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='3'>

In [7]:
client.search_model_versions()

[<ModelVersion: aliases=[], creation_timestamp=1718266398992, current_stage='None', description='', last_updated_timestamp=1718266398992, name='nyc-taxi-regressor', run_id='f783f99b110e448eb0d509b2f2d09953', run_link='', source='mlflow-artifacts:/1/f783f99b110e448eb0d509b2f2d09953/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='3'>,
 <ModelVersion: aliases=[], creation_timestamp=1718265542467, current_stage='Staging', description='', last_updated_timestamp=1718266124607, name='nyc-taxi-regressor', run_id='c409406a4bac42be9c14ab22f67c59a9', run_link='', source='mlflow-artifacts:/1/c409406a4bac42be9c14ab22f67c59a9/artifacts/model', status='READY', status_message='', tags={'model': 'GradientBoostingRegressor'}, user_id='', version='2'>,
 <ModelVersion: aliases=[], creation_timestamp=1718265497354, current_stage='Staging', description='', last_updated_timestamp=1718266113150, name='nyc-taxi-regressor', run_id='b4239ac133884f65936298bd9d2af4bc', run_link='

In [8]:
model_name = "nyc-taxi-regressor"
latest_versions = client.get_latest_versions(name=model_name)

for version in latest_versions:
    print(f"version: {version.version}, stage: {version.current_stage}")

version: 2, stage: Staging
version: 3, stage: None


In [9]:
model_version = 3
new_stage = "Staging"
client.transition_model_version_stage(
    name=model_name,
    version=model_version,
    stage=new_stage,
    archive_existing_versions=False
)

<ModelVersion: aliases=[], creation_timestamp=1718266398992, current_stage='Staging', description='', last_updated_timestamp=1718266639354, name='nyc-taxi-regressor', run_id='f783f99b110e448eb0d509b2f2d09953', run_link='', source='mlflow-artifacts:/1/f783f99b110e448eb0d509b2f2d09953/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='3'>

In [10]:
from datetime import datetime

date = datetime.today().date()
client.update_model_version(
    name=model_name,
    version=model_version,
    description=f"The model version {model_version} was transitioned to {new_stage} on {date}"
)

<ModelVersion: aliases=[], creation_timestamp=1718266398992, current_stage='Staging', description='The model version 3 was transitioned to Staging on 2024-06-13', last_updated_timestamp=1718266666421, name='nyc-taxi-regressor', run_id='f783f99b110e448eb0d509b2f2d09953', run_link='', source='mlflow-artifacts:/1/f783f99b110e448eb0d509b2f2d09953/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='3'>

Comparing versions and selecting the new "Production" model

In the last section, we will retrieve models registered in the model registry and compare their performance on an unseen test set. The idea is to simulate the scenario in which a deployment engineer has to interact with the model registry to decide whether to update the model version that is in production or not.

These are the steps:

1. Load the test dataset, which corresponds to the NYC Green Taxi data from the month of March 2021.

2. Download the DictVectorizer that was fitted using the training data and saved to MLflow as an artifact, and load it with pickle.

3. Preprocess the test set using the DictVectorizer so we can properly feed the regressors.

4. Make predictions on the test set using the model versions that are currently in the "Staging" and "Production" stages, and compare their performance.

5. Based on the results, update the "Production" model version accordingly.

In [11]:
from sklearn.metrics import mean_squared_error
import pandas as pd


def read_dataframe(filename):
    df = pd.read_parquet(filename)

    #df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    #df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df


def preprocess(df, dv):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    train_dicts = df[categorical + numerical].to_dict(orient='records')
    return dv.transform(train_dicts)

def test_model(name, stage, X_test, y_test):
    model = mlflow.pyfunc.load_model(f"models:/{name}/{stage}")
    y_pred = model.predict(X_test)
    return {"rmse": mean_squared_error(y_test, y_pred, squared=False)}

In [12]:
df = read_dataframe("data/green_tripdata_2021-03.parquet")


In [15]:
client.download_artifacts(run_id=run_id, path='preprocessor', dst_path='.')


'c:\\Users\\Aron\\Desktop\\mlops-zoomcamp\\02-experiment-tracking\\preprocessor'

In [16]:
import pickle

with open("preprocessor/preprocessor.b", "rb") as f_in:
    dv = pickle.load(f_in)

In [17]:
X_test = preprocess(df, dv)


In [18]:
target = "duration"
y_test = df[target].values

In [19]:
%time test_model(name=model_name, stage="Production", X_test=X_test, y_test=y_test)


CPU times: total: 3.61 s
Wall time: 10.4 s


c:\Users\Aron\anaconda3\envs\exp-tracking-env\lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


{'rmse': 6.659623830022513}

In [20]:
%time test_model(name=model_name, stage="Staging", X_test=X_test, y_test=y_test)


CPU times: total: 34.3 s
Wall time: 1min


c:\Users\Aron\anaconda3\envs\exp-tracking-env\lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


{'rmse': 6.874053955358958}

In [22]:
client.transition_model_version_stage(
    name=model_name,
    version=3,
    stage="Production",
    archive_existing_versions=True
)

<ModelVersion: aliases=[], creation_timestamp=1718266398992, current_stage='Production', description='The model version 3 was transitioned to Staging on 2024-06-13', last_updated_timestamp=1718270475121, name='nyc-taxi-regressor', run_id='f783f99b110e448eb0d509b2f2d09953', run_link='', source='mlflow-artifacts:/1/f783f99b110e448eb0d509b2f2d09953/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='3'>